In [ ]:
# ===============================================================
# Classificação Binária – Iris (todas as 4 features)
# Versicolor (1, classe positiva) vs Virginica (0, classe negativa)
# Modelos testados (sem árvore):
#   1) SVM (RBF)
#   2) SVM (Linear)
#   3) k-NN
#   4) MLP (rede neural)
#
# Saídas por modelo:
#   - métricas no TESTE (acc, precisão, recall, F1)
#   - matriz de confusão
#   - predições individuais (probabilidades quando disponíveis)
# ===============================================================

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, classification_report
)

RANDOM_STATE = 42
TEST_SIZE = 0.30

# ---------------------------------------------------------------
# 1) Carregar Iris e tornar BINÁRIA: 1=versicolor (positiva), 0=virginica
# ---------------------------------------------------------------
iris = load_iris()
X_full = iris.data            # (150, 4): sepal len, sepal wid, petal len, petal wid
y_full = iris.target          # 0=setosa, 1=versicolor, 2=virginica

mask = (y_full != 0)          # remove 'setosa'
X = X_full[mask]
y = (y_full[mask] == 1).astype(int)  # versicolor -> 1, virginica -> 0

feature_names = iris.feature_names

print("=== Dataset: Iris (binário) ===")
print("Classes: 0 = virginica (negativa), 1 = versicolor (positiva)")
print("Features (usaremos TODAS):", list(feature_names))
print(f"Instâncias totais: {X.shape[0]} | Nº de features: {X.shape[1]}\n")

# Amostra dos 10 primeiros registros (para os alunos verem a cara dos dados)
df_demo = pd.DataFrame(X, columns=feature_names)
df_demo["classe"] = np.where(y==1, "versicolor (1)", "virginica (0)")
print("=== 10 primeiras linhas ===")
print(df_demo.head(10).to_string(index=False))
print()

# ---------------------------------------------------------------
# 2) Split treino/teste (estratificado preserva a proporção das classes)
# ---------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)
print(f"Tamanhos -> treino: {X_train.shape[0]} | teste: {X_test.shape[0]}\n")

# ---------------------------------------------------------------
# 3) Definir os 4 modelos (todos com StandardScaler antes do classificador)
# ---------------------------------------------------------------
modelos = [
    ("SVM (RBF)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", gamma="scale", C=1.0,
                    probability=True, random_state=RANDOM_STATE))
    ])),
    ("SVM (Linear)", Pipeline([
        ("scaler", StandardScaler()),
        # LinearSVC não fornece probability; focamos em decisão por hiperplano
        ("clf", LinearSVC(C=1.0, random_state=RANDOM_STATE))
    ])),
    ("k-NN (k=5)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=5))
    ])),
    ("MLP (Rede Neural)", Pipeline([
        ("scaler", StandardScaler()),
        # uma camada escondida pequena (8) para didática, com early stopping
        ("clf", MLPClassifier(hidden_layer_sizes=(8,),
                              activation="relu",
                              solver="adam",
                              max_iter=1000,
                              random_state=RANDOM_STATE,
                              early_stopping=True))
    ])),
]

# ---------------------------------------------------------------
# 4) Funções utilitárias
# ---------------------------------------------------------------
def avaliar_modelo(nome, modelo, X_train, y_train, X_test, y_test):
    """
    Treina o modelo, faz previsões no TESTE e imprime:
    - Acurácia, Precisão (positiva=1), Recall (positiva=1), F1
    - Matriz de confusão
    Retorna o modelo treinado.
    """
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)   # linhas = Real, colunas = Previsto
    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, pos_label=1)
    rec  = recall_score(y_test, y_pred, pos_label=1)
    f1   = f1_score(y_test, y_pred, pos_label=1)

    print(f"=== {nome} ===")
    print(f"Acurácia: {acc:.3f}")
    print(f"Precisão (positiva=versicolor): {prec:.3f}")
    print(f"Recall   (positiva=versicolor): {rec:.3f}")
    print(f"F1       (positiva=versicolor): {f1:.3f}")
    print("\nMatriz de Confusão (linhas=Real, colunas=Previsto):")
    print("           Prev 0  Prev 1")
    print(f"Real 0  |   {cm[0,0]:>3}     {cm[0,1]:>3}   <- virginica (0)")
    print(f"Real 1  |   {cm[1,0]:>3}     {cm[1,1]:>3}   <- versicolor (1)\n")
    return modelo

def predicoes_individuais(nome, modelo, exemplos):
    """
    Mostra predições individuais com probabilidades quando o modelo oferece.
    """
    print(f"=== Predições Individuais — {nome} ===")
    if hasattr(modelo, "predict_proba"):
        probas = modelo.predict_proba(exemplos)   # colunas: [P(0), P(1)]
        preds  = modelo.predict(exemplos)
        for x, p, pred in zip(exemplos, probas, preds):
            print(f"Entrada {list(x)} -> prev={'versicolor (1)' if pred==1 else 'virginica (0)'} "
                  f"| P(virginica)= {p[0]:.3f} | P(versicolor)= {p[1]:.3f}")
    else:
        # Ex.: LinearSVC não tem predict_proba
        preds = modelo.predict(exemplos)
        for x, pred in zip(exemplos, preds):
            print(f"Entrada {list(x)} -> prev={'versicolor (1)' if pred==1 else 'virginica (0)'} "
                  f"(modelo não fornece probabilidade)")
    print()

# ---------------------------------------------------------------
# 5) Rodar os 4 modelos
# ---------------------------------------------------------------
treinados = []
for nome, mdl in modelos:
    mfit = avaliar_modelo(nome, mdl, X_train, y_train, X_test, y_test)
    treinados.append((nome, mfit))

    # Três exemplos com TODAS as 4 features (ordem: sepal len, sepal wid, petal len, petal wid)
    exemplos = np.array([
        [5.8, 2.8, 4.6, 1.4],  # tende a versicolor
        [6.3, 3.0, 5.8, 2.2],  # tende a virginica
        [6.0, 2.9, 4.5, 1.5]   # perto da fronteira
    ])
    predicoes_individuais(nome, mfit, exemplos)


=== Dataset: Iris (binário) ===
Classes: 0 = virginica (negativa), 1 = versicolor (positiva)
Features (usaremos TODAS): ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Instâncias totais: 100 | Nº de features: 4

=== 10 primeiras linhas ===
 sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)         classe
               7.0               3.2                4.7               1.4 versicolor (1)
               6.4               3.2                4.5               1.5 versicolor (1)
               6.9               3.1                4.9               1.5 versicolor (1)
               5.5               2.3                4.0               1.3 versicolor (1)
               6.5               2.8                4.6               1.5 versicolor (1)
               5.7               2.8                4.5               1.3 versicolor (1)
               6.3               3.3                4.7               1.6 versicolor (1)
             